# Set-up BW project

This nootebok sets the BW project, creating a new project, importing ecoinvent v3.10 and biosphere databases, and creating a database with regionalized water flows

In [1]:
import pandas as pd
import brightway2 as bw
from premise_gwp import add_premise_gwp
from pathlib import Path
import openpyxl
import bw2data as bd
from bw2io.utils import activity_hash
from copy import deepcopy

from utils import(
    create_biosphere_water_regionalized,
    create_aware_method
    )

## 1. Create new BW project and import ecoinvent v3.10

In [2]:
# Create new project
BW_PROJECT = 'lithium_brine'       # <-- insert project name here
bw.projects.set_current(BW_PROJECT)

In [16]:
# Import ecoinvent v3.10 and corresponding biosphere database
EI_USERNAME = "XXXXXXX"           # add ecoinvent credentials
EI_PASS = "YYYYYYYYYYYYY"         # add ecoinvent credentials

if "ecoinvent-3.10-cutoff" in bw.databases:
    print('Ecoinvent database already installed')
else:
    bw.import_ecoinvent_release(
        version="3.10", 
        system_model='cutoff', 
        username=EI_USERNAME,
        password=EI_PASS
        )

Ecoinvent database already installed


## 2. Import modified IPCC method to properly account for biogenic carbon
 see: https://github.com/polca/premise_gwp

 Note: the "mapping_ei310.yaml" file is not installed when installing with pip. I have downloaded it directly from the repository

In [ ]:
add_premise_gwp()

## 3. Create regionalized AWARE method

The created method contains site-specific CFs for lithium project and global CF for the rest of activities

In [3]:
# Get name of lithium projects and countries to add them to regionalized flow
# Projects names are containing the sheet name of each LCI excel file
lithium_project_names = []
lci_raw_path = Path("../inventories/raw")
lci_files = [file for file in lci_raw_path.glob("*.xlsx")]

for file in lci_files:
    wb = openpyxl.load_workbook(file, read_only=True)
    lithium_project_names.append(wb.sheetnames[0])

lithium_project_names

['Chaerhan',
 'East Taijinar',
 'Fenix',
 'Kachi',
 'Lakkor Tso',
 'Maricunga',
 'Pozuelos',
 'Qinghai Yiliping',
 'Sal de los Angeles',
 'Sal de Vida',
 'Salar de Antofalla',
 'Salar de Arizaro',
 'Salar de Atacama',
 'Salar de Cauchari-Olaroz',
 'Salar de Centenario',
 'Salar de Olaroz',
 'Salar de Pastos Grandes',
 'Salar de Rio Grande',
 'Salar de Salinas Grandes',
 'Salar de Tolillar',
 'Salar de Uyuni',
 'Salar del Hombre Muerto North',
 'Salar del Rincon',
 'Salton Sea',
 'Silver Peak',
 'Tres Quebradas',
 'Upper Rhine Graben',
 'Zhabuye']

In [4]:
# Create biosphere database with regionalized water flows
create_biosphere_water_regionalized(
        bio_db_name="ecoinvent-3.10-biosphere",
        reg_bio_db_name="biosphere water regionalized",
        locations=lithium_project_names
)

Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 01/21/2025 15:23:33
  Finished: 01/21/2025 15:23:33
  Total time elapsed: 00:00:00
  CPU %: 100.20
  Memory %: 2.34
biosphere water regionalized was created.


In [3]:
# Create AWARE method
create_aware_method(
    reg_bio_db_name="biosphere water regionalized",
    bio_db_name="ecoinvent-3.10-biosphere"
)

AWARE method created!


In [4]:
# Check the created regionalized CFs
AWARE_method = bw.Method(('AWARE regionalized', 'Annual', 'All')).load()

cf_list = []
for flow in AWARE_method:
    flow_key = flow[0]
    flow_cf = flow[1]

    print(flow_key)

('biosphere water regionalized', '5626a902fcdb0da20efde7f19d02568a')
('biosphere water regionalized', 'c3e77cf661f853a364fca513da8684ac')
('biosphere water regionalized', '911016941693cccdc5bcd85c4e1853ec')
('biosphere water regionalized', 'ad899c27e4953ec3cb56a46b0cf70c68')
('biosphere water regionalized', '82652e303048f9712af073a5f9b40cef')
('biosphere water regionalized', 'ec277eee35be47c381277ce9251056ae')
('biosphere water regionalized', 'ff15c9cd9ca935141fabe93e9c35961b')
('biosphere water regionalized', 'b56b909c32300edf25dda12ae85e78fd')
('biosphere water regionalized', 'e52f0706fb472373e224ee7ddc091dc5')
('biosphere water regionalized', '3cb303791b1a77757a2ff109a07157c3')
('biosphere water regionalized', 'c219286c20e35abd50a5b59029886146')
('biosphere water regionalized', 'aab11d06232ae16006ce6147b32c9705')
('biosphere water regionalized', 'd6902c30ff3ce1637e6e4fb9ddc25825')
('biosphere water regionalized', 'fa34a8d188531fba43d31e7b5a1066ca')
('biosphere water regionalized', '

In [5]:
# Check the created regionalized CFs
AWARE_method = bw.Method(('AWARE regionalized', 'Annual', 'All')).load()

cf_list = []
for flow in AWARE_method:
    flow_key = flow[0]
    flow_cf = flow[1]

    if flow_key[0] == "biosphere water regionalized":
        flow_ds = [flow for flow in bw.Database("biosphere water regionalized") if flow.key==flow_key]
        cf_list.append(((flow_ds[0]["name"], flow_ds[0]["categories"]), flow_ds[0]["location"], flow_cf))
    elif flow_key[0] == "ecoinvent-3.10-biosphere":
        flow_ds = [flow for flow in bw.Database("ecoinvent-3.10-biosphere") if flow.key==flow_key]
        cf_list.append(((flow_ds[0]["name"], flow_ds[0]["categories"]), "GLO", flow_cf))
   # if len(flow_ds)==1:
   #     cf_list.append(((flow_ds[0]["name"], flow_ds[0]["categories"]), flow_ds[0]["location"], flow_cf))
        
cf_list_df = pd.DataFrame(cf_list, columns=["name", "location", "cf"])
cf_list_df = cf_list_df.groupby(["location", "name"], as_index=False).mean()
cf_list_df = cf_list_df.pivot(index="location", columns="name", values="cf").reset_index()

In [6]:
cf_list_df

name,location,"(Water, (air,))","(Water, (air, low population density, long-term))","(Water, (air, lower stratosphere + upper troposphere))","(Water, (air, non-urban air or from high stacks))","(Water, (air, urban air close to ground))","(Water, irrigation, (air,))","(Water, irrigation, (air, low population density, long-term))","(Water, irrigation, (air, lower stratosphere + upper troposphere))","(Water, irrigation, (air, non-urban air or from high stacks))","(Water, irrigation, (air, urban air close to ground))"
0,Chaerhan,70.600000,70.600000,70.600000,70.600000,70.600000,48.500000,48.500000,48.500000,48.500000,48.500000
1,Fenix,2.700000,2.700000,2.700000,2.700000,2.700000,2.700000,2.700000,2.700000,2.700000,2.700000
2,GLO,42.953531,42.953531,42.953531,42.953531,42.953531,NaN,NaN,NaN,NaN,NaN
3,Kachi,4.800000,4.800000,4.800000,4.800000,4.800000,0.600000,0.600000,0.600000,0.600000,0.600000
4,Lakkor Tso,92.600000,92.600000,92.600000,92.600000,92.600000,92.600000,92.600000,92.600000,92.600000,92.600000
5,Maricunga,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000
6,Pozuelos,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000
7,Qinghai Yiliping,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000
8,Sal de Vida,2.700000,2.700000,2.700000,2.700000,2.700000,2.700000,2.700000,2.700000,2.700000,2.700000
9,Sal de los Angeles,2.700000,2.700000,2.700000,2.700000,2.700000,2.700000,2.700000,2.700000,2.700000,2.700000
